# Deckbuilding Interactive Testing

This notebook provides interactive testing for the deckbuilding functionality in the statistical drafting library.


In [1]:
# Install the package in development mode
%pip install -e .. -q

  DEPRECATION: Legacy editable install of statisticaldrafting==0.0.1 from file:///Users/danielbrooks/Desktop/Code/statistical-drafting (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import statisticaldrafting as sd

## Load Sample Game Data

Let's load some game data to test our deckbuilding functions.


In [3]:
# Load game data - adjust path as needed (first 10,000 rows for testing)
data_path = "../data/games/game_data_public.FDN.PremierDraft.csv.gz"
df_game = pd.read_csv(data_path, compression="gzip", nrows=10000)
print(f"Loaded {len(df_game)} game records")
df_game.head()


Loaded 10000 game records


expansion    event_type                          draft_id  \
0       FDN  PremierDraft  c296734752c7449592905c452b88688f   
1       FDN  PremierDraft  f4060a8ab54f4a02b0916f3b0d984141   
2       FDN  PremierDraft  f4060a8ab54f4a02b0916f3b0d984141   
3       FDN  PremierDraft  690d05dbbf9940f695bb1169553dfca6   
4       FDN  PremierDraft  a65901b9b2c943a0ab4215b5c7d7a591   

            draft_time            game_time  build_index  match_number  \
0  2024-11-12 19:09:18  2024-11-12 19:39:12            0             1   
1  2024-11-12 21:20:12  2024-11-12 21:45:34            0             1   
2  2024-11-12 21:20:12  2024-11-12 23:00:21            3             2   
3  2024-11-14 01:58:01  2024-11-14 02:16:27            0             1   
4  2024-11-17 21:56:47  2024-11-17 22:16:23            0             1   

   game_number      rank  opp_rank  ... tutored_Zombify deck_Zombify  \
0            1  platinum       NaN  ...               0            0   
1            1       NaN       NaN  ...               0            0   
2            1  platinum       NaN  ...               0            0   
3            1  platinum       NaN  ...               0            0   
4            1  platinum       NaN  ...               0            0   

   sideboard_Zombify  opening_hand_Zul Ashur, Lich Lord  \
0                  0                                  0   
1                  0                                  0   
2                  0                                  0   
3                  0                                  0   
4                  0                                  0   

   drawn_Zul Ashur, Lich Lord tutored_Zul Ashur, Lich Lord  \
0                           0                            0   
1                           0                            0   
2                           0                            0   
3                           0                            0   
4                           0                            0   

   deck_Zul Ashur, Lich Lord  sideboard_Zul Ashur, Lich Lord  \
0                          0                               0   
1                          0                               0   
2                          0                               0   
3                          0                               0   
4                          0                               0   

   user_n_games_bucket  user_game_win_rate_bucket  
0                  100                       0.54  
1                  100                       0.54  
2                  100                       0.54  
3                  100                       0.54  
4                  100                       0.54  

[5 rows x 1450 columns]

## Test ETL Function

Test the `etl_game()` function to extract relevant deckbuilding data.


In [4]:
# Test the etl_game function
processed_df = sd.etl_game(df_game)
print(f"Original records: {len(df_game)}")
print(f"Processed records: {len(processed_df)}")
print(f"Columns before: {len(df_game.columns)}")
print(f"Columns after: {len(processed_df.columns)}")


Original records: 10000
Processed records: 1684
Columns before: 1450
Columns after: 592


## Test Dataset Creation

Test the `create_deckbuild_dataset()` function to create training data for deckbuilding models.


In [6]:
# Test creating a deckbuilding dataset (using small sample for testing)
# First let's create a small sample dataset to test with
sample_size = 1000
df_sample = df_game.head(sample_size)

# Test the dataset creation with a small sample
print(f"Testing with {len(df_sample)} game records")

# Test basic removal
print(f"Columns before basic removal: {len(df_sample.columns)}")
df_sample_no_basics = sd.remove_basics_from_games(df_sample)
print(f"Columns after basic removal: {len(df_sample_no_basics.columns)}")

# Apply ETL to see the structure
sample_processed = sd.etl_game(df_sample_no_basics)
print(f"After ETL: {len(sample_processed)} records")

# Look at the deck and sideboard columns
deck_cols = [col for col in sample_processed.columns if col.startswith("deck_")]
sideboard_cols = [col for col in sample_processed.columns if col.startswith("sideboard_")]

print(f"Found {len(deck_cols)} deck columns")
print(f"Found {len(sideboard_cols)} sideboard columns")

# Check if any basic lands remain
basic_names = ["Forest", "Island", "Mountain", "Plains", "Swamp"]
remaining_basics = [col for col in deck_cols if any(basic in col for basic in basic_names)]
if remaining_basics:
    print(f"Warning: Found remaining basic columns: {remaining_basics}")
else:
    print("✅ All basic lands successfully removed")

# Show some example card counts in decks
if len(deck_cols) > 0:
    print("\nSample deck composition (first few cards):")
    for col in deck_cols[:5]:
        card_name = col[5:]  # Remove "deck_" prefix
        total_copies = sample_processed[col].sum()
        print(f"  {card_name}: {total_copies} total copies across all decks")
    
    print(f"\nDeck size statistics:")
    deck_sizes = sample_processed[deck_cols].sum(axis=1)
    print(f"  Mean deck size: {deck_sizes.mean():.1f}")
    print(f"  Std deck size: {deck_sizes.std():.1f}")
    print(f"  Min/Max deck size: {deck_sizes.min()}/{deck_sizes.max()}")


Testing with 1000 game records
Columns before basic removal: 1450
Removed 25 basic land columns
Columns after basic removal: 1425
After ETL: 163 records
Found 281 deck columns
Found 281 sideboard columns
✅ All basic lands successfully removed

Sample deck composition (first few cards):
  Abrade: 18 total copies across all decks
  Abyssal Harvester: 5 total copies across all decks
  Adventuring Gear: 0 total copies across all decks
  Aegis Turtle: 6 total copies across all decks
  Aetherize: 1 total copies across all decks

Deck size statistics:
  Mean deck size: 24.5
  Std deck size: 1.3
  Min/Max deck size: 23/28


In [7]:
# Test the create_deckbuild_dataset function with a limited sample
# Note: This would normally process the full dataset, but we'll test the functionality

# For a real test, you would call:
# train_path, val_path = sd.create_deckbuild_dataset("FDN", "Premier", overwrite=True)

# Instead, let's test the data processing logic manually to verify it works
print("Testing deckbuilding dataset creation logic...")

# Simulate the key steps of create_deckbuild_dataset
df_test = sample_processed.copy()

# Filter for good players (similar to what the function does)
min_winrate = df_test["user_n_games_bucket"].apply(sd.get_min_winrate, p=0.40, stdev=1.5) # reduced p for testing
df_filtered = df_test[df_test["user_game_win_rate_bucket"] >= min_winrate]
print(f"After filtering for good players: {len(df_filtered)} records")

if len(df_filtered) > 0:
    # Get deck and sideboard data
    deck_data = df_filtered[sorted(deck_cols)].astype(int)
    sideboard_data = df_filtered[sorted(sideboard_cols)].astype(int)
    
    # Create pool vectors (deck + sideboard)
    pool_data = deck_data.values + sideboard_data.values
    deck_vectors = deck_data.values
    
    print(f"Pool data shape: {pool_data.shape}")
    print(f"Deck data shape: {deck_vectors.shape}")
    
    # Test creating a small DeckbuildDataset
    cardnames = [col[5:] for col in sorted(deck_cols)]  # Remove "deck_" prefix
    
    if len(df_filtered) >= 2:  # Need at least 2 samples for dataset
        test_dataset = sd.DeckbuildDataset(pool_data[:2], deck_vectors[:2], cardnames)
        print(f"Created test dataset with {len(test_dataset)} examples")
        
        # Test accessing dataset items
        pool_sample, deck_sample = test_dataset[0]
        print(f"Sample pool vector shape: {pool_sample.shape}")
        print(f"Sample deck vector shape: {deck_sample.shape}")
        print(f"Sample pool sum (total cards available): {pool_sample.sum().item()}")
        print(f"Sample deck sum (cards in main deck): {deck_sample.sum().item()}")
        
        print("✅ Dataset creation logic works correctly!")
    else:
        print("Not enough filtered samples to test dataset creation")
else:
    print("No records passed the filtering criteria")


Testing deckbuilding dataset creation logic...
After filtering for good players: 159 records
Pool data shape: (159, 281)
Deck data shape: (159, 281)
Created test dataset with 2 examples
Sample pool vector shape: torch.Size([281])
Sample deck vector shape: torch.Size([281])
Sample pool sum (total cards available): 42.0
Sample deck sum (cards in main deck): 24.0
✅ Dataset creation logic works correctly!


In [8]:
for x, y in test_dataset:
    print(x.shape, y.shape)

torch.Size([281]) torch.Size([281])
torch.Size([281]) torch.Size([281])


In [8]:
x.sum(), y.sum()

(tensor(41.), tensor(26.))

## Test Deckbuilding Training Logic

Test the core training sample creation and model training logic.


In [10]:
# Test the training sample creation logic
print("Testing deckbuild training sample creation...")

import torch

# Use the test dataset we created earlier
if 'test_dataset' in locals() and len(test_dataset) > 0:
    # Get a sample pool and deck
    pool_sample, deck_sample = test_dataset[0]
    
    print(f"Original pool sum: {pool_sample.sum().item()}")
    print(f"Original deck sum: {deck_sample.sum().item()}")
    
    # Test creating training samples
    current_deck, available_cards, target_card = sd.create_deckbuild_training_sample(
        pool_sample, deck_sample
    )
    
    print(f"Current deck sum (after removing card): {current_deck.sum().item()}")
    print(f"Available cards sum: {available_cards.sum().item()}")
    print(f"Target card sum: {target_card.sum().item()}")
    print(f"Target card index: {torch.argmax(target_card).item()}")
    
    # Verify the math: current_deck + available_cards should equal pool
    reconstructed_pool = current_deck + available_cards
    print(f"Reconstructed pool sum: {reconstructed_pool.sum().item()}")
    print(f"Pool reconstruction matches: {torch.allclose(pool_sample, reconstructed_pool)}")
    
    print("✅ Training sample creation logic works!")
else:
    print("No test dataset available. Run the previous cells first.")


Testing deckbuild training sample creation...
Original pool sum: 42.0
Original deck sum: 24.0
Current deck sum (after removing card): 23.0
Available cards sum: 19.0
Target card sum: 1.0
Target card index: 134
Reconstructed pool sum: 42.0
Pool reconstruction matches: True
✅ Training sample creation logic works!


In [ ]:
# Test creating and evaluating a small deckbuild model
print("Testing deckbuild model training...")

if 'test_dataset' in locals() and len(test_dataset) >= 4:
    from torch.utils.data import DataLoader
    
    # Create small test dataloaders with batch size >= 2 to avoid BatchNorm issues
    train_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)  # Use batch_size=2 for BatchNorm
    
    # Create a small model for testing
    cardnames = test_dataset.cardnames
    test_network = sd.DraftNet(cardnames=cardnames, dropout_input=0.1)
    
    print(f"Created test network with {len(cardnames)} cards")
    
    # Test evaluation (before training)
    print("Testing evaluation function...")
    initial_accuracy = sd.evaluate_deckbuild_model(val_loader, test_network)
    print(f"Initial accuracy: {initial_accuracy:.2f}%")
    
    # Test a few training steps (not full training)
    print("Testing training function (2 epochs only)...")
    
    # Temporarily modify the training function for quick testing
    import time
    import torch.optim as optim
    
    loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
    optimizer = optim.Adam(test_network.parameters(), lr=0.01)
    
    for epoch in range(2):  # Just 2 epochs for testing
        test_network.train()
        epoch_loss = []
        
        for pool_batch, deck_batch in train_loader:
            batch_size = pool_batch.shape[0]
            
            for i in range(batch_size):
                pool_vector = pool_batch[i]
                deck_vector = deck_batch[i]
                
                # Create training sample
                current_deck, available_cards, target_card = sd.create_deckbuild_training_sample(
                    pool_vector, deck_vector
                )
                
                # Forward pass
                optimizer.zero_grad()
                current_deck_input = current_deck.unsqueeze(0).float()
                available_cards_input = available_cards.unsqueeze(0).float()
                target_input = target_card.unsqueeze(0).float()
                
                predicted_card = test_network(current_deck_input, available_cards_input)
                loss = loss_fn(predicted_card, target_input).mean()
                
                loss.backward()
                optimizer.step()
                epoch_loss.append(loss.item())
        
        print(f"Epoch {epoch}: Loss = {np.mean(epoch_loss):.4f}")
    
    # Test evaluation after training
    final_accuracy = sd.evaluate_deckbuild_model(val_loader, test_network)
    print(f"Final accuracy: {final_accuracy:.2f}%")
    
    print("✅ Deckbuild model training logic works!")
    
else:
    print("Need at least 4 samples in test_dataset for training test")


Testing deckbuild model training...
Need at least 4 samples in test_dataset for training test


## Full Model Training (Optional)

Uncomment and run the cell below to train a complete deckbuilding model on the full dataset.


In [12]:
# Uncomment to train a full deckbuilding model
# WARNING: This will take significant time and computational resources

training_info = sd.default_deckbuild_training_pipeline(
    set_abbreviation="FDN",
    draft_mode="Premier", 
    overwrite_dataset=False,  # Use existing dataset
    dropout_input=0.6
)

print("Training completed!")
print(f"Final validation accuracy: {training_info['validation_accuracy']:.2f}%")
print(f"Model saved as: {training_info['experiment_name']}.pt")

# The model will be saved as "FDN_Premier_deckbuild.pt" to avoid overwriting draft models

# print("Full model training is commented out. Uncomment the code above to run.")


Deckbuilding training and validation sets already exist. Skipping.
Starting deckbuild model training. learning_rate=0.03
Deckbuild validation accuracy = 6.1%

Starting epoch 0  lr=0.03


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 400])

## Full Dataset Creation (Optional)

Uncomment and run the cell below to create the full deckbuilding dataset. This will process all game data and create training/validation sets.


In [ ]:
# Uncomment to create the full deckbuilding dataset
# WARNING: This will process the entire game dataset and may take some time
import torch

train_path, val_path = sd.create_deckbuild_dataset(
    set_abbreviation="FDN",
    draft_mode="Premier",
    overwrite=True,
    data_folder_games="../data/games/",
    data_folder_cards="../data/cards/"
)

# print(f"Training dataset saved to: {train_path}")
# print(f"Validation dataset saved to: {val_path}")

# Test loading the created dataset
train_dataset = torch.load(train_path, weights_only=False)
val_dataset = torch.load(val_path, weights_only=False)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

# Test a sample from the training set
sample_pool, sample_deck = train_dataset[0]
print(f"Sample shapes - Pool: {sample_pool.shape}, Deck: {sample_deck.shape}")

# print("Full dataset creation is commented out. Uncomment the code above to run.")


Using input file ../data/games/game_data_public.FDN.PremierDraft.csv.gz
Loading game data...
